## Libraries

#### Python Integrated or in Folder

In [54]:
import ipywidgets as widgets
import thinkdsp as thinkdsp
from thinkdsp import play_wave
from thinkdsp import read_wave
import math 
import soundfile as sf
import pyloudnorm as pyln
import pydub 
from pydub import AudioSegment
import spectrum

#### Require Installation

##### Enter these commands into your terminal:
    pip install soundfile
    pip install pyloudnorm
    pip install pydub
    pip install spectrum

## Variables

#### NOT Recommended for change

In [2]:
ChooseBandCount = 10 # Standard Band Count
Controllrange = [-12, 12, 0] # [Decrease, Gain, Default] in dB
EQBandmin = 5
EQBandmax = 20
EQstep = 5
ConfigBandCount = 21

In [3]:
Freqspecturmf0 = [16,32,64,128,256,512,1024,2048,4096,8192,0,0,0,0,0,0,0,0,0,0,0] # Displayed Frequency on EQ 
Freqspecturmfl = [0]*EQBandmax # Lower Frequency for EQ
Freqspecturmfh = [0]*EQBandmax # Higher Frequency for EQ

In [4]:
# Generates Grid for EQ
# EQDisplay = widgets.GridspecLayout(ChooseBandCount, 1)

#### Recommended for change

In [5]:
Text1 = widgets.Output()
with Text1:
    print("Choose your File, that you want to edit:")
    
Text2 = widgets.Output()
with Text2:
    print("\nChoose the focus of the EQ bandpass:")
    
Text3 = widgets.Output()
with Text3:
    print("\nChoose the EQ Slider Count:")
    
Text4 = widgets.Output()
with Text4:
    print("\nEqualizer:")
    
Text5 = widgets.Output()
with Text5:
    print("\nFinished Audio File:")

In [6]:
UploadFileName = "Upload"
TempFileName = "TempFile"
FileFormat = ".wav"
FinishedFileName = "Output.wav"
CloneFileName = "Band"

## Functions

##### Generates EQ UI

In [7]:
def Build_EQ (ChooseBandCount, Freqspecturmf0):
    
    slider = [0] * ChooseBandCount
    
    for i in range(ChooseBandCount):
        slider[i] = widgets.IntSlider(
           value=Controllrange[2],
            min=Controllrange[0],
            max=Controllrange[1],
            step=1,
            description=str(Freqspecturmf0[i])+' Hz',
            disabled=False,
            continuous_update=False,
            orientation='vertical',
            readout=True,
            readout_format='d',
            layout = widgets.Layout(width='50%')
        )
        
        # print(slider[i].value)
    
    # Box = widgets.HBox(slider)
    # EQDisplay = Accordian(children=[Box])
    
    # EQDisplay = widgets.HBox(slider)
    
    # EQDisplay = widgets.Box(children = slider)
    
    # display(EQDisplay)
    return slider
    # return slider

In [8]:
# EQDisplay = Build_EQ(ChooseBandCount, Freqspecturmf0)

In [9]:
# def Test (EQDisplay):
#   print(EQDisplay[0].value)
#   print("Test")

In [10]:
# EQDisplay = Build_EQ(ChooseBandCount, Freqspecturmf0)
# ShowEQDisplay = widgets.Box(children = EQDisplay)

# display(ShowEQDisplay)
#Equalizer = widgets.interact(Test, EQDisplay = widgets.fixed(EQDisplay))
#Equalizer = widgets.interactive_output(Test, {'EQDisplay': widgets.fixed(ShowEQDisplay)})

In [11]:
# print(EQDisplay.slider[0].value)

In [12]:
# SaveValue = [0]*ChooseBandCount

# for i in range(ChooseBandCount):
#    SaveValue[i] = EQDisplay[i].value

# print(SaveValue)

##### Calculating Base Frequency of choosen Type of EQ 

In [13]:
def returnbasefreq(SetFilterType, ChooseBandCount):
    # Scans input of choosen Type of EQ and gives back predefined start Frequencies for an 8 Band EQ
    if (SetFilterType == 'Bass'):
        f_base = 20
    if (SetFilterType == 'Mid'):
        f_base = 40
    if (SetFilterType == 'High'):
        f_base = 80
    
    # Modulates Frequency depending on difference between 8 / ChoosenBand
    f_base = f_base * (8/ChooseBandCount)
        
    return f_base

##### Calculates Base Frequencies that are displayed on EQ

In [14]:
def calculate_frequencies(SetFilterType, ChooseBandCount):
    #resets values in Freqspecturm
    Freqspecturmf0 = [0] * (EQBandmax+1) # Clears all values that were precalculatet
    
    # Select start frequency for calculation
    f_base = int(returnbasefreq(SetFilterType, ChooseBandCount))
    # Calculates Modulation Rate for n-Frequencies
    f_mod = pow(2, 10 / ChooseBandCount)
    
    # Generates Freqspecturm for given Type and Bandcount
    Freqspecturmf0[0] = f_base
    
    for i in range(1, ChooseBandCount+1):
        Freqspecturmf0[i] = int(f_mod * Freqspecturmf0[i-1]) # int() Rounds Value to non decimal
    
    # print (Freqspecturm)
    return Freqspecturmf0

# SetFilterType = 'Mid'
# ChooseBandCount = 20
# calculate_frequencies(SetFilterType, ChooseBandCount)

##### Calculates log average of f0[n] and f0[n-1]

In [15]:
def calculate_frequencieslow(Freqspecturmf0, ChooseBandCount):
    # For the value of fl[0] of f0[0] we need to calculate f0[-1], fortunatly because we dont need a highpass on the first EQ we can set fl[0] = 0
    Freqspecturmfl = [0]*EQBandmax # Clears all values that were precalculatet
    
    Freqspecturmfl[0] = 0
    
    for i in range(1, ChooseBandCount):
            Freqspecturmfl[i] = int(math.sqrt(Freqspecturmf0[i] * Freqspecturmf0[i-1]))
    
    return Freqspecturmfl

##### Calculates log average of f0[n] and f0[n+1]

In [16]:
def calculate_frequencieshigh(Freqspecturmf0, ChooseBandCount):
    # For the value of fh[n] of f0[n] we need to calculate f0[n+1]
    Freqspecturmfh = [0]*EQBandmax # Clears all values that were precalculatet
    
    f_mod = pow(2, 10 / ChooseBandCount)
    fnp1 = f_mod * Freqspecturmf0[ChooseBandCount]
    
    for i in range(0, ChooseBandCount):
            Freqspecturmfh[i] = int(math.sqrt(Freqspecturmf0[i] * Freqspecturmf0[i+1]))
    
    return Freqspecturmfh

##### Normalising Audio

In [17]:
# Value is the dB Value that the file will be normalised with
# void function: Overrides [FileName] File

def Normalise_Audio (FileName, FileFormat, Normalise):
    # Load Audio File as shape(data, wave)
    data, rate = sf.read("Upload.wav")

    # Initiolising loudness Meter by BS.1770 standart 
    meter = pyln.Meter(rate) 
    # measure loudness in dB
    loudness = meter.integrated_loudness(data)
    
    # loudness normalize audio to [Normalise] dB LUFS
    loudness_normalized_audio = pyln.normalize.loudness(data, loudness, Normalise)
    sf.write(FileName + FileFormat, loudness_normalized_audio, rate)

##### Generating temporary files

In [18]:
# Generates Temp Copies for Processing every single Band
# File Name is the UploadFile Name
# FileName must not contail ".wav" !!!

def Generate_Copies(FileName, FileFormat, BandpassCount):
    # import Upload File
    wave = AudioSegment.from_wav("Upload.wav")
    
    for i in range(0, ChooseBandCount):
        wave.export(CloneFileName + str(i) + FileFormat, "wav")

##### Bandpass Filter 

In [81]:
# Overwrites [n] temp Audio File
# Filename is the current edited Bandpass
# Gain is the current used Gain value of said bandpass

def Bandpass_Filter (FileName, FileFormat, n, Gain, fl, fh):
    # Load Audio File
    wave = read_wave(FileName + str(n) + FileFormat)
    spectrum = wave.make_spectrum()
    # Lowpass filter with fh
    spectrum.low_pass(fh)
    # Highpass filter with fl
    spectrum.high_pass(fl)
    # Overrides Temp Audio File
    #with open(FileName + str(n) + FileFormat, "w+b") as edit:
    #    edit.write(spectrum)
    wave.write(FileName + str(n) + FileFormat)
    
    
    
    # Change Gain
    song = AudioSegment.from_wav(FileName + str(n) + FileFormat)

    # but let's make him *very* quiet
    song_10_db_quieter = song - Gain

    # save the output
    song_10_db_quieter.export(FileName + str(n) + FileFormat, "wav")
    

##### Build Finished Audio

In [73]:
# Overlaps several AudioFiles which have the same base name + and adressing Number + ".wav"
# Example "FilterTemp" + string(n) + ".wav" starting from 0 to Bandpasscount-1
# FileName must not contail ".wav" !!!
# FileType is the file ending, for example ".wav"
# FinishedFileName is the Filename which the Audio will be saved. Must contain Filename+".wav"

def Build_Audio (FileName, FileFormat, FinishedFileName, BandpassCount):
    # Load first Adio file
    fullaudio = AudioSegment.from_wav(FileName + "0" + FileFormat)
    # Generates loop
    for i in range(1, ChooseBandCount - 2):
        # import current file
        temp = AudioSegment.from_wav(FileName + str(i) + FileFormat)
        # Overlays fullaudio with new generated wave file
        fullaudio = fullaudio.overlay(temp)

    # Saves fullaudio
    fullaudio.export("Output.wav", "wav")
    
    # Deletes temp Files (optional)

## Widgets

In [21]:
# Upload a file
#Upload = widgets.FileUpload()

Widget_Upload = widgets.FileUpload(
    accept= FileFormat,  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    tooltips=['Upload a .wav File to Edit it']
)

In [22]:
Widget_Verify_Upload = widgets.ToggleButton(
    value=False,
    description='Submit',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [23]:
Widget_Verify_EQ = widgets.ToggleButton(
    value=False,
    description='Calculate',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [24]:
# Choose EQ Preset
Widget_SetFilterType = widgets.ToggleButtons(
    options=['Bass', 'Mid', 'High'],
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Focus on 200Hz', 'Focus on 1kHz', 'Focus on 4kHz']
)

In [25]:
# Choose Band Count
Widget_ChooseBandCount = widgets.BoundedIntText(
    value=ChooseBandCount,
    min=EQBandmin,
    max=EQBandmax,
    step=EQstep,
    disabled=False
)

In [26]:
# Generates Base EQ
# EQDisplay = Build_EQ (EQBandmax, Freqspecturmf0)

# display(EQDisplay[2])

#### Combine all Widgets

In [27]:
UserInterface = widgets.VBox(children = [Text1, Widget_Upload, Widget_Verify_Upload, Text2, Widget_SetFilterType, Text3, Widget_ChooseBandCount, Text4])

In [28]:
# display(UserInterface)

## Main Calculation

#### Saves Uploaded File and resets Widget

In [29]:
# Triggered when: An file is uploaded

def SaveUpload(Widget_Verify_Upload):
    # Safe Uploaded File
    with open(UploadFileName + FileFormat, "w+b") as i:
        i.write(Widget_Upload.data[0])
    
    Widget_Upload.value.clear()
    Widget_Upload._counter = 0
    print("Upload Sucessfull!")

#### Configuration Calculation

In [30]:
# Triggered when: BandCount or FilterType is changed
ChoosenBandCount = ChooseBandCount
EQDisplay = Build_EQ (20, Freqspecturmf0)

def ProcessConfiguration(SetFilterType, ChooseBandCount):
    # Calculate Base Frequency
    f_base = returnbasefreq(SetFilterType, ChooseBandCount)
    # Calculate f0 Frequencies
    global Freqspecturmf0
    Freqspecturmf0 = calculate_frequencies(SetFilterType, ChooseBandCount)
    # Calculates f0l
    global Freqspecturmfl
    Freqspecturmfl = calculate_frequencieslow(Freqspecturmf0, ChooseBandCount)
    # Calculates f0h
    global Freqspecturmfh
    Freqspecturmfh = calculate_frequencieshigh(Freqspecturmf0, ChooseBandCount)
    
    global ChoosenBandCount
    ChoosenBandCount = ChooseBandCount
    
    # Overwrites EQ Data
    global EQDisplay
    EQDisplay = Build_EQ (ChooseBandCount, Freqspecturmf0)
    ShowEQDisplay = widgets.Box(children = EQDisplay)
    display(ShowEQDisplay)
    
    # SaveValue = [0]*ChooseBandCount

    # for i in range(ChooseBandCount):
    #    SaveValue[i] = EQDisplay[i].value 

    # print(SaveValue)
    # display("Test")

#### EQ Calculation

In [79]:
# Triggered 

def EQCalculation(Widget_Verify_EQ):
    # Normalises Audio to -MaxGain - 3dB, that the maximum Output Amplification -3db is reached
    Normalise = Controllrange[0] - 3
    Normalise_Audio(UploadFileName, FileFormat, Normalise)
    
    Generate_Copies (UploadFileName, FileFormat, ChoosenBandCount)
    
    # Get EQ Values
    SaveValue = [0]*ChoosenBandCount
    for i in range(ChoosenBandCount):
        SaveValue[i] = EQDisplay[i].value
    
    
    # Value of EQ Display is the Gain Value
    for n in range(ChoosenBandCount):
        Bandpass_Filter (CloneFileName, FileFormat, n, SaveValue[n], Freqspecturmfl[n], Freqspecturmfh[n])
    
    Build_Audio (CloneFileName, FileFormat, FinishedFileName, ChoosenBandCount)
    
    
    wave = read_wave("Output.wav")
    plot = wave.make_audio()
    display(plot)

## User Interface

In [84]:
if __name__ == "__main__":
    Upload = widgets.interactive_output(SaveUpload, {'Widget_Verify_Upload' : Widget_Verify_Upload})
    Configuration = widgets.interactive_output(ProcessConfiguration, {'SetFilterType' : Widget_SetFilterType, 'ChooseBandCount' : Widget_ChooseBandCount})
    # ActivateEqualizer = widgets.interactive_output(EQCalculation, {'Widget_Verify_EQ' : Widget_Verify_EQ})
    
    # Equalizer = widgets.interactive_output(EQCalculation, {'EQ_Display' : EQDisplay})
    
    
    # Temp 
    
    # GetValue = EQDisplay.value
    # print(GetValue)
    # ShowEQDisplay = widgets.Box(children = EQDisplay)
    
    # display(UserInterface, Configuration, Text5)
    display(UserInterface, Configuration, Text5, Widget_Verify_Upload)
    
    # ToDo List
    
    # 1.) Finish Audio Processing (80%, needs final testing after UI is finished)
    # 2.) Get EQDisplay Values from widgets.Box(children = slider) (10%, dont know what the problem is lol)
    # 3.) Output finished File and Plot (95%, plot needs to be added, needs to be decided what type of plot is used or if any plot is even needed)

Output()

Output(outputs=({'name': 'stdout', 'text': '\nFinished Audio File:\n', 'output_type': 'stream'},))

ToggleButton(value=True, button_style='success', description='Submit', icon='check')

In [90]:
Normalise = Controllrange[0] - 3

Normalise_Audio(UploadFileName, FileFormat, Normalise)
    
Generate_Copies (UploadFileName, FileFormat, ChoosenBandCount)
    
# Get EQ Values
SaveValue = [0]*ChoosenBandCount
for i in range(ChoosenBandCount):
    SaveValue[i] = EQDisplay[i].value
    
    
# Value of EQ Display is the Gain Value
for n in range(ChoosenBandCount):
    Bandpass_Filter (CloneFileName, FileFormat, n, SaveValue[n], Freqspecturmfl[n], Freqspecturmfh[n])
    
Build_Audio (CloneFileName, FileFormat, FinishedFileName, ChoosenBandCount)
    
    
wave = read_wave("Output.wav")
plot = wave.make_audio()
display(plot)

Writing Band0.wav
Writing Band1.wav
Writing Band2.wav
Writing Band3.wav
Writing Band4.wav
Writing Band5.wav
Writing Band6.wav
Writing Band7.wav
Writing Band8.wav
Writing Band9.wav


In [33]:
SaveValue = [0]*ChoosenBandCount

for i in range(ChoosenBandCount):
    SaveValue[i] = EQDisplay[i].value

print(SaveValue)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Formulas Used

## Sources

#### Example Audio File Credits
    Song: Dimension 
    Creator: Creo 
    Youtube: https://www.youtube.com/channel/UCsCWA3Y3JppL6feQiMRgm6Q 
    Website: https://creo-music.com/track/dimension
    Licensed under: https://creativecommons.org/licenses/by/4.0/

#### Programming
    https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
    https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
    https://en.wikibooks.org/wiki/LaTeX/Mathematics

#### Formulars and Documents for Audio
    https://www.teachmeaudio.com/mixing/techniques/audio-spectrum#upper-midrange
    https://sound.stackexchange.com/questions/14101/what-is-the-frequency-step-formula-for-10-and-31-band-eqs
    http://www.sengpielaudio.com/calculator-octave.html
    https://github.com/AllenDowney/ThinkDSP

#### Other Programms used
    Music editing: Audacity
    Music Download: https://github.com/yt-dlp/yt-dlp

#### Special Thanks
    https://www.senarclens.eu/~gerald/

## Programmer

#### Programmed by Thomas Pail
    GitHub: https://github.com/Tomaru-Pai